In [ ]:
import sys
from google.colab import files
sys.path.insert(0,'/content/drive/MyDrive/IR-Project-Colab')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/IR-Project-Colab/Utilities.py /content 
!cp /content/drive/MyDrive/IR-Project-Colab/KNNImpute.py /content

In [ ]:
import Utilities  #THIS IS FOR GOOGLE COLAB USERS ONLY
import KNNImpute

In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


from sklearn.feature_extraction.text import TfidfVectorizer   # Turning textual data into numeric for computation
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder               # For encoding categorical target attr
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

from sklearn.tree import DecisionTreeClassifier
from sklearn import svm   # Baseline
from sklearn.svm import SVC

# ------- Validation metrics
from sklearn.metrics import accuracy_score    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import hamming_loss                  
from sklearn.metrics import classification_report



### ***READING CSV AND KNN IMPUTE***

In [ ]:
nltk.download('stopwords')
import pandas as pd
labels=["country", "sku_id", "title", "category_lvl1","category_lvl2","category_lvl3", "description", "price", "type"]

def getTrainingDataset():
    return pd.read_csv('data_train.csv', header=None, names=labels)  


def NullStatistics(df):
    missing_val = df.isnull().sum()
    print(missing_val)
    total_cells = np.product(df.shape)
    missing_percent = (missing_val.sum()/total_cells) *100
    print(f'\nThe missing data percent is: {missing_percent}')

trainingDataset=getTrainingDataset()

train_df,Y1,Y2,Y3=Utilities.Cleaning_Data_Utility(trainingDataset) #Utilities.

NullStatistics(train_df)

unique_label_c1, unique_label_c2, unique_label_c3 = KNNImpute.preserve_label(train_df) #It has all unique values lying in ctg1 , 2 , 3 column

KNNImpute.encode(['category_lvl1', 'category_lvl2', 'category_lvl3'],train_df)        #Performed encoding for CTGLVL3 KNN

train_df_imputed = KNNImpute.impute(train_df)
train_df_imputed =  KNNImpute.clean_csv(train_df_imputed,train_df)
NullStatistics(train_df_imputed)

In [ ]:
print(len(unique_label_c1))
print(len(unique_label_c2))
print(len(unique_label_c3))

### ***FEATURE ENGINEERING***

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer   # Turning textual data into numeric for computation
X = train_df_imputed['Title_desc']
Y1 = train_df_imputed['category_lvl1']
Y2 = train_df_imputed['category_lvl2']
Y3 = train_df_imputed['category_lvl3']

vectorizer = TfidfVectorizer()  #Only once

vectorizer.fit(X)
X_tfidf = vectorizer.transform(X)
# print(X_tfidf)

### ***MODEL TRAINING***

### *CATEGORY* 1 

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# X_tfidf.reshape(-1,1)
# X_tfidf
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y1, test_size=0.3, random_state=42)
print (X_train.shape)
print (X_test.shape)

SVM_Classfier=SVC(kernel='linear' , random_state=0)
model1 = SVM_Classfier.fit(X_train,y_train)
Y_Pred_c1=model1.predict(X_test)


print(Y_Pred_c1)
90.47

### *CATEGORY 1 STATISTICS*

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

print('Accuracy score with linear kernel is : ')
print(metrics.accuracy_score(y_test,Y_Pred_c1)*100)
cf_matrix = confusion_matrix(y_test, Y_Pred_c1)
print(classification_report(y_test, Y_Pred_c1))

plt.figure(figsize=(44,12))

sns.heatmap(cf_matrix,annot=True ,fmt='d',  cmap='Blues',square=True ,cbar=True, linewidths=.10, xticklabels= unique_label_c1, yticklabels= unique_label_c1  )
plt.ylabel('Actual lable')
plt.xlabel('Predicted lable')

### *CATEGORY 2*

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y2, test_size=0.3, stratify= Y2, random_state=42)


SVM_Classfier=SVC(kernel='linear' , random_state=0)
model2=SVM_Classfier.fit(X_train,y_train)
Y_Pred_c2=model2.predict(X_test)
print(Y_Pred_c2)


### *CATEGORY 2 STATISTICS*

In [ ]:
print('Accuracy score with linear kernel is : ')
print(metrics.accuracy_score(y_test,Y_Pred_c2)*100)

cf_matrix = confusion_matrix(y_test, Y_Pred_c2)
print(classification_report(y_test, Y_Pred_c2))

plt.figure(figsize=(44,12))

sns.heatmap(cf_matrix,annot=True ,fmt='d',  cmap='Blues',square=True ,cbar=True, linewidths=.10, xticklabels= unique_label_c2, yticklabels= unique_label_c2  )
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

### *CATEGORY 3*

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y3, test_size=0.3, stratify= Y3, random_state=42)

SVM_Classfier=SVC(kernel='linear' , random_state=0)
model3=SVM_Classfier.fit(X_train,y_train)
Y_Pred_c3=model3.predict(X_test)



### *CATEGORY 3 STATISTICS*

In [ ]:
print('Accuracy score with linear kernel is : ')
print(metrics.accuracy_score(y_test,Y_Pred_c3)*100)

cf_matrix = confusion_matrix(y_test, Y_Pred_c3)
print(classification_report(y_test, Y_Pred_c3))

plt.figure(figsize=(70,70))

sns.heatmap(cf_matrix,annot=True ,fmt='d',  cmap='Blues',square=True ,cbar=True, linewidths=.10, xticklabels= unique_label_c3, yticklabels= unique_label_c3  )
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

### ***MODEL TESTING***

*DATA VALID CSV READ*

In [ ]:

labels=["country", "sku_id", "title", "category_lvl1","category_lvl2","category_lvl3", "description", "price", "type"]

def NullStatistics(df):
    missing_val = df.isnull().sum()
    print(missing_val)
    total_cells = np.product(df.shape)
    missing_percent = (missing_val.sum()/total_cells) *100
    print(f'\nThe missing data percent is: {missing_percent}')

testingDataset=pd.read_csv('data_valid.csv' , header=None, names=labels) 

train_df,Y1,Y2,Y3=Utilities.Cleaning_Data_Utility(testingDataset) 

NullStatistics(train_df)

unique_label_c1, unique_label_c2, unique_label_c3 = KNNImpute.preserve_label(train_df) 

KNNImpute.encode(['category_lvl1', 'category_lvl2', 'category_lvl3'],train_df)      

train_df_imputed = KNNImpute.impute(train_df)
train_df_imputed =  KNNImpute.clean_csv(train_df_imputed,train_df)
NullStatistics(train_df_imputed)

In [ ]:
#Printing Unique Labels in Txt Files.


*FEATURE ENGINEERING*

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer   # Turning textual data into numeric for computation
X = train_df_imputed['Title_desc']
Y1 = train_df_imputed['category_lvl1']
Y2 = train_df_imputed['category_lvl2']
Y3 = train_df_imputed['category_lvl3']

# vectorizer = TfidfVectorizer()
# vectorizer.fit(X)   #Test (Unseen data is never fitted)
x_test_tfidf = vectorizer.transform(X)



### CATEGORY *1* PREDICTION

In [ ]:
y_pred_cat_one = model1.predict(x_test_tfidf)


CATEGORY 1 STATISTICS

In [ ]:
print(metrics.accuracy_score(Y1, y_pred_cat_one)*100)
cf_matrix = confusion_matrix(Y1, y_pred_cat_one)
print(classification_report(Y1, y_pred_cat_one))

plt.figure(figsize=(44,12))

sns.heatmap(cf_matrix,annot=True ,fmt='d',  cmap='Blues',square=True ,cbar=True, linewidths=.10, xticklabels= unique_label_c1, yticklabels= unique_label_c1  )
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

### *CATEGORY 2*

In [ ]:
y_pred_cat_2 = model2.predict(x_test_tfidf)


CATEGORY 2 STATISTICS

In [ ]:
print(metrics.accuracy_score(Y2, y_pred_cat_2)*100)
cf_matrix = confusion_matrix(Y2, y_pred_cat_2)
print(classification_report(Y2, y_pred_cat_2))

plt.figure(figsize=(44,12))

sns.heatmap(cf_matrix,annot=True ,fmt='d',  cmap='Blues',square=True ,cbar=True, linewidths=.10, xticklabels= unique_label_c2, yticklabels= unique_label_c2  )
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

### CATEGORY *3*

In [ ]:
y_pred_cat_3 = model3.predict(x_test_tfidf)
print(metrics.accuracy_score(Y3, y_pred_cat_3)*100)

CATEGORY 3 STATISTICS

In [ ]:
print(metrics.accuracy_score(Y3, y_pred_cat_3)*100)
cf_matrix = confusion_matrix(Y3, y_pred_cat_3)
print(classification_report(Y3, y_pred_cat_3))

plt.figure(figsize=(44,12))

sns.heatmap(cf_matrix,annot=True ,fmt='d',  cmap='Blues',square=True ,cbar=True, linewidths=.10, xticklabels= unique_label_c3, yticklabels= unique_label_c3  )
plt.ylabel('Actual lable')
plt.xlabel('Predicted lable')

In [ ]:
def decode_cat01(number):
    le =LabelEncoder()
    le.fit(unique_label_c1)
    LabelEncoder()
    # print(list(le.classes_))  # print to check classes
    le.transform(unique_label_c1)

    return str(le.inverse_transform([number]))

def decode_cat02(number):
    le =LabelEncoder()
    le.fit(unique_label_c2)
    LabelEncoder()
    # print(list(le.classes_))  # print to check classes
    le.transform(unique_label_c2)

    return str(le.inverse_transform([number]))

def decode_cat03(number):
    le =LabelEncoder()
    le.fit(unique_label_c3)
    LabelEncoder()
    # print(list(le.classes_))  # print to check classes
    le.transform(unique_label_c3)

    return str(le.inverse_transform([number]))
    

print(decode_cat01(1))
print(decode_cat02(29))
print(decode_cat03(98))


### ***MODEL SAVE***

In [ ]:
import pickle
pickle.dump(model1, open('model1.pickle', 'wb'))
pickle.dump(model2, open('model2.pickle', 'wb'))
pickle.dump(model3, open('model3.pickle', 'wb'))


In [ ]:
pickle.dump(model3, open('model3.pickle', 'wb'))


In [ ]:
os.listdir('.')

In [ ]:
pickle.dump(vectorizer, open("vectorizer.pickle", "wb"))

### ***QUERY TESTING***

In [ ]:
def Load_Svm_Models():
    model_c1_svm = pickle.load(open('model1.pickle', 'rb'))
    model_c2_svm = pickle.load(open('model2.pickle', 'rb'))
    model_c3_svm = pickle.load(open('model3.pickle', 'rb'))
    return model_c1_svm, model_c2_svm, model_c3_svm

# # def Load_Vectorizer():
def Predict_Query_SVM(query):
    tfidf_vectorizer = pickle.load(open('vectorizer.pickle','rb'))
    corpus_vocabulary = defaultdict(None, copy.deepcopy(tfidf_vectorizer.vocabulary_))
    corpus_vocabulary.default_factory = corpus_vocabulary.__len__
    m1,m2,m3=Load_Svm_Models()
    tfidf_transformer_query = TfidfVectorizer()
    tfidf_transformer_query.fit_transform(query)

    for word in tfidf_transformer_query.vocabulary_.keys():
        if word in tfidf_vectorizer.vocabulary_:
            corpus_vocabulary[word]

    tfidf_transformer_query_sec = TfidfVectorizer(vocabulary=corpus_vocabulary)
    query_tfidf_matrix = tfidf_transformer_query_sec.fit_transform(query)
    return m1.predict(query_tfidf_matrix), m2.predict(query_tfidf_matrix),m3.predict(query_tfidf_matrix)


In [ ]:
query="Laptop"
query=Utilities.PreProcessing(query)
print(query)
query=[query]

m1,m2,m3=Predict_Query_SVM(query)
Ctg1=decode_cat01(int(m1))
Ctg2=decode_cat02(int(m2))
Ctg3=decode_cat03(int(m3))
print(Ctg1,Ctg2,Ctg3)


In [ ]:
# from collections import defaultdict
# import copy

# corpus_vocabulary = defaultdict(None, copy.deepcopy(vectorizer.vocabulary_))
# corpus_vocabulary.default_factory = corpus_vocabulary.__len__

# tfidf_transformer_query = TfidfVectorizer()
# tfidf_transformer_query.fit_transform(query)

# for word in tfidf_transformer_query.vocabulary_.keys():
#     if word in vectorizer.vocabulary_:
#         corpus_vocabulary[word]

# tfidf_transformer_query_sec = TfidfVectorizer(vocabulary=corpus_vocabulary)
# query_tfidf_matrix = tfidf_transformer_query_sec.fit_transform(query)

# Category_1_Output=int(model1.predict(query_tfidf_matrix))
# Category_2_Output=int(model2.predict(query_tfidf_matrix))

# Category_3_Output=int(model3.predict(query_tfidf_matrix))

# Ctg1=decode_cat01(Category_1_Output)
# Ctg2=decode_cat02(Category_2_Output)
# Ctg3=decode_cat03(Category_3_Output)

# print(Ctg1,Ctg2,Ctg3)

